Create a Neural Network from Scratch


In [36]:
import numpy as np

class Layer:
    def __init__(self):
        self.input = None
        self.output = None
    
    def forward(self, input):
        self.layer_input = input
        self.output = np.dot(input, self.weights) + self.biases
        return self.output

    def backward(self, grad_output, lr):
        # TODO update parameters and return input gradient
        pass
    
class Dense(Layer):
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(input_size, output_size) * 0.01
        self.biases = np.zeros(output_size)

    def forward(self, input):
        self.input = input
        self.output = np.dot(input, self.weights) + self.biases
        return self.output

    def backward(self, grad_output, lr):
       # TODO update parameters and return input gradient
        pass
    
class Activation(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    def forward(self, input):
        self.input = input
        return self.activation(self.input)
    
    def backward(self, output_gradient, learning_rate):
        # TODO update parameters and return input gradient
        pass
        
class Tanh(Activation):
    def __init__(self):
        tanh = lambda x: np.tanh(x)
        tanh_prime = lambda x: 1 - np.tanh(x) ** 2
        super().__init__(tanh, tanh_prime)

def mse(y_true, y_pred):
    return np.mean(np.power(y_true - y_pred, 2))

def mse_prime(y_true, y_pred):
    return 2 * (y_pred - y_true) / np.size(y_true)


In [87]:
def test():
    # Create a test input
    x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
    # Define a test output
    y = np.array([[0], [1], [1], [0]])

    # Create a simple neural network
    dense1 = Dense(2, 3)
    activation1 = Tanh()
    dense2 = Dense(3, 1)
    activation2 = Tanh()

    # Forward pass
    output1 = dense1.forward(x)
    output2 = activation1.forward(output1)
    output3 = dense2.forward(output2)
    y_pred = activation2.forward(output3)

    # Print the output
    print("Predicted output:")
    print(y_pred)

    # Calculate the loss
    loss = mse(y, y_pred)
    print("Loss:")
    print(loss)

# Run the test
test()


Predicted output:
[[0.        ]
 [0.00022224]
 [0.00018512]
 [0.00040731]]
Loss:
0.4997963868550465
